In [42]:
import feast # must be imported first, not sure why.
import quspin as qp
print(qp.__version__)

0.3.4


In [52]:
import feast # must be imported first, not sure why.
import quspin as qp
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d,boson_basis_1d # Hilbert space boson basis
import numpy as np # generic math| functions
import time
from scipy.linalg import eig

start=time.time()
##### define model parameters #####
L = 10 # system size
Nb=L//2
# sps=4
J = 1.0 # hopping strength
halfU = 1.0

basis=boson_basis_1d(L=L, Nb=Nb)
print(basis.Ns)

U_n=[[-halfU,i] for i in range(L)]
J_nn=[[J,i,i+1] for i in range(L-1)]
J_nn2=[[J+0.2,i,i+1] for i in range(L-1)]
U_int=[[halfU,i,i] for i in range(L)]
static=[["+-",J_nn],["-+",J_nn2],["nn",U_int],["n",U_n]]

###### construct Hamiltonian
H_b=hamiltonian(static,[],dtype=np.float64,basis=basis,check_herm=False)
H=H_b.static
spec,psil=eig((H_b.static).todense())
print("scipy.linalg.eig finished!")


### Working for real or complex general csr_matrix (Compressed Sparse Row matrix)
### M0:subspace size; Emid:Coordinate center of the contour ellipse; r:Horizontal radius of the contour ellipse;
### Search eigenvalues less than r from Emid in the complex plane, largest/lowest not supported.
### which:1(right eigenvalues), 0(both); default which=1
### methods for GSolver class:
### .fpm: access fpm array
### .setfpm(cp=None,eps=None,ml=None,it=None,ra=None): set fpm array
### .eigs(debug=0): run FEAST, only right eigenvalues/vectors will returned.
### .eig(): access right eigenvlaues
### .eigl(): access left eigenvalues
### .vec(): access right eigenvectors
### .vecl(): access left eigenvectors
### .resid(): access residual of right eigenvlaues
### .residl(): access residual of left eigenvlaues

which=0
obj=feast.GSolver(H,M0=40,Emid=-8.0,r=1.0,which=which) # find right eigenvalues in (E+9.0)^2 <= r^2
obj.setfpm(cp=40,eps=12,ml=2,it=0) # change default fpm parameter if needed
# M,info=obj.eigs(se=2) # estimate of #eigenvalues, M0>=1.5M is suggested, change Em/Ex or rebuild obj if M0<1.5M.
egs,psi,M,info=obj.eigs(debug=0)
if info == 0:
    print("FEAST sucess!",M,"eigenvlaues found.")
elif info == 6:
    print("FEAST warning!",M,"eigenvlaues found, subspace is not orthonormal")
else:
    print("info =",info,"FEAST warning or failed!")
# info Classiﬁcation Description
# 202 Error Problem with size of the system N
# 201 Error Problem with size of subspace M0
# 200 Error Problem with Emin, Emax or Emid, r
# (100 + i) Error Problem with ithvalue of the input FEAST parameter (i.e fpm(i))
# 7 Warning The search for extreme eigenvalues has failed, search contour must be set by user
# 6 Warning FEAST converges but subspace is not bi-orthonormal
# 5 Warning Only stochastic estimation of #eigenvalues returned fpm(14)=2
# 4 Warning Only the subspace has been returned using fpm(14)=1
# 3 Warning Size of the subspace M0 is too small (M0<=M)
# 2 Warning No Convergence (#iteration loops>fpm(4))
# 1 Warning No Eigenvalue found in the search interval
# 0 Successful exit
# −1 Error Internal error conversion single/double
# −2 Error Internal error of the inner system solver in FEAST Driver interfaces
# −3 Error Internal error of the reduced eigenvalue solverPossible cause for Hermitian problem: matrix B may not be positive deﬁnite
# −(100 + i) Error Problem with the ithargument of the FEAST interface

2002
Symmetry checks passed!
Particle conservation check passed!
scipy.linalg.eig finished!
FEAST sucess! 12 eigenvlaues found.


In [2]:
# possible input for .setfpm function
# cp: contour points, it=0,cp in (2 to 40, 48, 64, 80, 96, 112), it=1, any cp>2
# eps: convergence criteria = 10^(-eps) 
# ml: refinement loop, ml>0
# it: integration type, 0: Gauss; 1: Trapezoidal;
# vh: contour ratio ’vertical axis’/’horizontal axis’, ratio=vh/100
# ra: contour rotation angle in degree from vertical axis ra in [-180:180]
# lsp: mixed precision, 0:double; 1: single
# ifeast: automatic switch if ifeast=1, usually used when matrix is very large (>1M)
# ifp: accuracy of ifeast, 10^(-ipf)
# ifl: loop of ifeast
# ref Table 1 @ http://www.ecs.umass.edu/~polizzi/feast/doc.htm
obj.setfpm(cp=40,eps=12,ml=2,it=1,vh=100,ra=0,lsp=1,ifeast=0,ifp=1,ifl=40,Emid=-8.0,r=1.0)
obj.fpm

array([     0,      8,     12,      8,      0,      1,      5,     40,
         -111,      1,   -111,   -111,      0,      0,      0,      0,
         -111,    100,      0,     40,    100,     40,     38,      1,
           38,   2002,   -111,   -111,      1, 121342,     40,     10,
           40,     41,     38,      1,      0,      1,      0,      0,
            1,      1,      0,      0,      1,     40,      0,      0,
            0,   -111,   -111,   -111,   -111,   -111,   -111,   -111,
         -111,   -111,   -111,      0,   -111,   -111,   -111,      0],
      dtype=int32)

In [53]:
# parameters after FEAST finished
# [1:real/2:complex, M0, Emid, r, 0:both/1:right_only, loop, espout]
obj.para_check()

[1, 38, (-8+0j), 1.0, 0, 2, 2.8957725501162154e-15]

In [54]:
# right normalized
np.array([psi[:,i].conj().dot(psi[:,i]) for i in range(M)])

array([0.70174161+0.j, 0.89234073+0.j, 1.80129574+0.j, 1.48162419+0.j,
       1.48511592+0.j, 1.95669542+0.j, 2.04516653+0.j, 2.16346921+0.j,
       1.26092253+0.j, 2.29769581+0.j, 2.29612732+0.j, 2.48012993+0.j])

In [55]:
# bi-normalized
if which==0:
    pl=obj.vecl()
    print(np.array([pl[:,i].conj().dot(psi[:,i]) for i in range(M)]))

[1.-1.11022302e-16j 1.+1.11022302e-16j 1.+1.66533454e-16j
 1.+0.00000000e+00j 1.+0.00000000e+00j 1.+5.55111512e-17j
 1.+0.00000000e+00j 1.-5.55111512e-17j 1.+5.55111512e-17j
 1.-1.11022302e-16j 1.+0.00000000e+00j 1.-8.32667268e-17j]


In [56]:
# those are eigenvectors
print(np.array([sum(abs(H.dot(psi[:,i])-egs[i]*psi[:,i])) for i in range(M)]))
if which==0:
    print(np.array([sum(abs(H.conj().transpose().dot(pl[:,i])-egs[i].conj()*pl[:,i])) for i in range(M)]))

[5.75233695e-12 1.06697411e-11 1.03067258e-11 3.54201356e-12
 1.50825524e-11 3.82564237e-12 1.41854376e-11 8.70803420e-12
 9.61948499e-12 7.82263596e-12 4.13926658e-12 6.72479725e-12]
[6.11726094e-12 3.83612202e-12 5.86334100e-12 4.08068816e-12
 1.03382575e-11 2.63679889e-12 4.05686331e-12 2.45343513e-12
 4.27702851e-12 3.74836982e-12 2.21321113e-12 1.67773693e-12]


In [57]:
# compare with scipy.linalg.eigh
np.array([sum(abs(H.dot(psil[:,i])-spec[i]*psil[:,i])) for i in range(M)])

array([1.14464015e-12, 1.22905581e-12, 1.36680294e-12, 1.49882567e-12,
       1.53173090e-12, 1.65699563e-12, 1.59447434e-12, 1.64369878e-12,
       1.03334117e-12, 1.03140969e-12, 1.84868459e-12, 1.75695641e-12])

In [58]:
# compare eigenvalues from FEAST with scipy.linalg.eigh
abs(np.sort(egs)-np.sort(spec)[:M])

array([1.03177378e-13, 4.63409365e-14, 6.04106458e-14, 5.07218570e-14,
       1.96886732e-14, 2.57703854e-14, 6.05522559e-14, 7.63834110e-14,
       4.42639511e-15, 2.22785840e-14, 6.21825152e-14, 3.65278296e-14])

In [59]:
# residual of right eigenvalues
obj.resid()

array([2.59584214e-14, 4.23620524e-14, 2.88022825e-14, 1.08422228e-14,
       4.66203047e-14, 1.04845738e-14, 3.77635328e-14, 2.23861291e-14,
       3.27175175e-14, 1.90799657e-14, 1.05844409e-14, 1.65317582e-14])

In [60]:
# residual of left eigenvalues
obj.residl()

array([1.64395980e-14, 9.10200963e-15, 1.92026462e-14, 1.26069926e-14,
       3.14854061e-14, 7.50582414e-15, 1.16659541e-14, 9.64520827e-15,
       1.43754383e-14, 1.42715784e-14, 6.51521244e-15, 6.52366436e-15])